# Azure AI Agents function calling

NOTE: If this is your first time, you need to authenticate your API requests. 

Use the az login command to sign into your Azure subscription. 

In Terminal run: az login

In [ ]:
%pip install azure-ai-projects
%pip install azure-identity

## Define a function for your agent to call
Start by defining a function for your agent to call. When you create a function for an agent to call, you describe its structure of it with any required parameters in a docstring.

In [ ]:
import json
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def fetch_restaurant(location: str) -> str:
    """
    Fetches the restaurant information for the specified location.

    :param location (str): The location to fetch the restaurant for.
    :return: Restaurant information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a restaurant API.
    # Here, we'll mock the response.
    mock_restaurant_data = {
        "New York": "Tatiana by Kwame Onwuachi, Katz’s Delicatessen, Peter Luger Steakhouse, Sylvia's, Nathan's Famous", 
        "London": "St. JOHN, Señor Ceviche, Gloria and Circolo Popolare, Normah's, Bouchon Racine", 
        "Tokyo": "Chanko & Wanko Restaurant Asakusa Sumo Club, Sky Restaurant 634 Musashi, Ichiran, Shibuya, Rokkasen Otakibashiidori, Hakushu - Kobe Teppanyaki"
    }
    restaurant = mock_restaurant_data.get(location, "Restaurant data not available for this location.")
    restaurant_json = json.dumps({"restaurant": restaurant})
    return restaurant_json

def fetch_budget() -> str:
    """
    Fetches the budget information for the specified location.
    :return: budget information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a another API.
    # Here, we'll mock the response.
    mock_budget_data = {
        "New York": """
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "London": """
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "Tokyo": """
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
    }
    budget_json = json.dumps({"budget": mock_budget_data})
    return budget_json


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather, fetch_restaurant, fetch_budget
}

## STEP 1: Create a client and agent

In the sample below we create a client and define a toolset which will be used to process the functions defined in user_functions.

toolset: When using the toolset parameter, you provide not only the function definitions and descriptions but also their implementations. The SDK will execute these functions within create_and_run_process or streaming. These functions will be invoked based on their definitions.

In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o", 
    name="my-agent", 
    instructions="You are a weather bot. Use the provided functions to help answer questions.", 
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

## STEP 2: Create a thread

In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

## Step 3-6: Helper Function
3. Add a message to the thread
4. Run the Agent
5. Check the Run Status
6. Display the Agent's Response


In [88]:
def run_agent(user_input):  
    # Step 3: Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Step 4: Run the agent
    run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Created run, ID: {run.id}")

    # Step 5: Check the Run Status
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        # Print the current status of the run
        print(f"Current run status: {run.status}")

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

        # Step 6: Display the Agent's Response
        elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread.id)
            if messages.data:
                agent_message = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_message.content[0].text.value}") 
            else:
                print("No messages found.")

## Running the agent using the Helper Function

In [89]:
user_input = "Hello, what is the weather in Tokyo?"
run_agent(user_input)

Created message, ID: msg_2Th635KwpISwsjntodVuqnu7
Created run, ID: run_YOSLntbNmiRhIQGPb8MlTWFS
Current run status: RunStatus.REQUIRES_ACTION
Executing tool call: {'id': 'call_ZMZjJQQ3qBH4ZYNV7iF0JbAS', 'type': 'function', 'function': {'name': 'fetch_weather', 'arguments': '{"location":"Tokyo"}'}}
Tool outputs: [{'tool_call_id': 'call_ZMZjJQQ3qBH4ZYNV7iF0JbAS', 'output': '{"weather": "Rainy, 22\\u00b0C"}'}]
Current run status: RunStatus.COMPLETED
Agent Response: The current weather in Tokyo is rainy, with a temperature of 22°C.


In [90]:
user_input = "What is the budget there?"
run_agent(user_input)

Created message, ID: msg_LMIHpwb9YfPa7GnCbvCxQm1G
Created run, ID: run_oqNsVn2mhQrFEyKthzl8K3O3
Current run status: RunStatus.REQUIRES_ACTION
Executing tool call: {'id': 'call_1KFF71Ti4MRGb2TmGeBxI9hq', 'type': 'function', 'function': {'name': 'fetch_budget', 'arguments': '{}'}}
Tool outputs: [{'tool_call_id': 'call_1KFF71Ti4MRGb2TmGeBxI9hq', 'output': '{"budget": {"New York": "\\n            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.\\n            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.\\n            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.\\n        ", "London": "\\n            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.\\n            Mid-Range Traveler

In [92]:
user_input = "Where can I eat in London?"
run_agent(user_input)

Created message, ID: msg_h0gwZiDjBqUQngijBiNeipjR
Created run, ID: run_E7fgDVwhI5SHk9MteLiRFGtO
Current run status: RunStatus.REQUIRES_ACTION
Executing tool call: {'id': 'call_5trPmB6db51SNe1MBlZ1FMRk', 'type': 'function', 'function': {'name': 'fetch_restaurant', 'arguments': '{"location":"London"}'}}
Tool outputs: [{'tool_call_id': 'call_5trPmB6db51SNe1MBlZ1FMRk', 'output': '{"restaurant": "St. JOHN, Se\\u00f1or Ceviche, Gloria and Circolo Popolare, Normah\'s, Bouchon Racine"}'}]
Current run status: RunStatus.COMPLETED
Agent Response: In London, you can dine at:

- **St. JOHN**
- **Señor Ceviche**
- **Gloria and Circolo Popolare**
- **Normah's**
- **Bouchon Racine**


In [93]:
user_input = "I only have 300 USD. What country can I visit for 4 days?"
run_agent(user_input)

Created message, ID: msg_Q1YnQq16jeNUeTXpFf65KAQb
Created run, ID: run_5hCGoEqgS754UziNrzMbbe6J
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Agent Response: Based on your $300 budget for 4 days, here's a breakdown of what countries you could afford to visit, considering the daily expenses provided earlier:

- **London:** For budget travelers, it's $75 per day. In 4 days, your total cost would be **$300**, fitting exactly within your budget. You would need to stick to hostels, cook meals, and rely on public transportation.

- **Tokyo:** For budget travelers, it's $100 per day, which totals to **$400** for 4 days. Tokyo would exceed your budget unless you can reduce expenses.

- **New York:** For budget travelers, it's $121 per day, which totals to **$484** for 4 days. New York would exceed your budget.

Given these options, **London** is the best choice for your budget.


## Delete Agent to free up resources

In [94]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
